In [143]:
import os
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [144]:
channel_id = 'UCsqjHFMB_JYTaEnf_vmTNqg'
api_key = os.environ['YOUTUBE_API_KEY']

# print(api_key)

## Extraction

In [145]:
def channel_stats(api_key, channel_id):
    channel_stats_url = f"https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2Cstatistics&id={channel_id}&key={api_key}"
    # print(channel_stats_url)
    response = requests.get(channel_stats_url)
    channel_data = json.loads(response.text)
    # print(channel_data)
    return channel_data

In [146]:
def get_channel_videos(api_key,channel_id):
    page_count = 0
    url = f"https://youtube.googleapis.com/youtube/v3/search?channelId={channel_id}&maxResults=2&order=date&type=video&key={api_key}"


    def get_channel_videos_per_page(url):
        channel_videos = []
    
        response = requests.get(url)
        data = json.loads(response.text)
        nextPageToken = data.get('nextPageToken')
        for i in range(len(data['items'])):
            videoId = data['items'][i]['id']['videoId']
            channel_videos.append(videoId)

        return nextPageToken, channel_videos

    nextPageToken, channel_videos = get_channel_videos_per_page(url)

    while nextPageToken and page_count < 5:
        next_page_url = f"{url}&pageToken={nextPageToken}"
        nextPageToken, new_channel_videos = get_channel_videos_per_page(next_page_url)
        channel_videos.extend(new_channel_videos)
        page_count += 1

    # print(channel_videos)
    return channel_videos

# get_channel_videos(api_key, channel_id)


In [147]:

def get_video_stats(api_key):
    videos_stats = []
    videoIds = get_channel_videos(api_key, channel_id)

    for videoId in videoIds:
        video_data= {}
        video_stats_url = f"https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2Cstatistics&id={videoId}&key={api_key}"
        response = requests.get(video_stats_url)
        data = json.loads(response.text)

        video_data = {
            'publishedAt' : data['items'][0]['snippet']['publishedAt'],
            'title' : data['items'][0]['snippet']['title'],
            'views' : data['items'][0]['statistics']['viewCount'],
            'likes' : data['items'][0]['statistics']['likeCount'],
            'comments' : data['items'][0]['statistics']['commentCount'],

        }
        videos_stats.append(video_data)



        # print(videoId)
    print(videos_stats)


# get_video_stats(api_key)



In [148]:
channel_data = channel_stats(api_key,channel_id)
video_data = get_video_stats(api_key)

KeyError: 'items'

In [ ]:
channel_data

## Transform

In [149]:
channel_stats ={
        'channel_id': channel_data['items'][0]['id'],
        'channel_title' : channel_data['items'][0]['snippet']['title'],
        'publishedAt' : channel_data['items'][0]['snippet']['publishedAt'],
        'subscribers' : channel_data['items'][0]['statistics']['subscriberCount'],
        'views' : channel_data['items'][0]['statistics']['viewCount'],
        'videos' : channel_data['items'][0]['statistics']['videoCount']
}

df = pd.DataFrame([channel_stats])

# change datatypes
df['views'] = df['views'].astype(int)
df['videos'] = df['videos'].astype(int)
df['subscribers'] = df['subscribers'].astype(int)
df['publishedAt'] = dt.datetime.strptime(df['publishedAt'].str.split('T')[0][0], '%Y-%m-%d').strftime('%Y-%m-%d')
df

,channel_id,channel_title,publishedAt,subscribers,views,videos
0,UCsqjHFMB_JYTaEnf_vmTNqg,Doug DeMuro,2013-09-11,4440000,1632917605,801


## Load

In [161]:
df.to_csv(f"{channel_stats['channel_title']}.csv", index = False)
df['channel_title']

0    Doug DeMuro
Name: channel_title, dtype: object